In [11]:
import os 
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [12]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader 
from langchain_text_splitters import MarkdownTextSplitter

In [7]:
loader = UnstructuredMarkdownLoader('../mmd/Chapter_01.mmd', mode="elements")
data = loader.load()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shayn\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\shayn\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [8]:
data

[Document(page_content='Signals and Systems\n\nIn this chapter we shall discuss basic aspects of signals and systems. We shall also introduce fundamental concepts and qualitative explanations of the hows and whys of systems theory, thus building a solid foundation for understanding the quantitative analysis in the remainder of the book. For simplicity, the focus of this chapter is on continuous-time signals and systems. Chapter 3 presents the same ideas for discrete-time signals and systems.\n\nSignals\n\nA signal is a set of data or information. Examples include a telephone or a television signal, monthly sales of a corporation, or daily closing prices of a stock market (e.g., the Dow Jones averages). In all these examples, the signals are functions of the independent variable time. This is not always the case, however. When an electrical charge is distributed over a body, for instance, the signal is the charge density, a function of space rather than time. In this book we deal almost

In [9]:
# split using Markdown rules
markdown_splitter = MarkdownTextSplitter(chunk_size=500, chunk_overlap=0)
split_docs = markdown_splitter.split_documents(data)

In [10]:
split_docs

[Document(page_content='Signals and Systems\n\nIn this chapter we shall discuss basic aspects of signals and systems. We shall also introduce fundamental concepts and qualitative explanations of the hows and whys of systems theory, thus building a solid foundation for understanding the quantitative analysis in the remainder of the book. For simplicity, the focus of this chapter is on continuous-time signals and systems. Chapter 3 presents the same ideas for discrete-time signals and systems.\n\nSignals', metadata={'source': '../mmd/Chapter_01.mmd'}),
 Document(page_content='A signal is a set of data or information. Examples include a telephone or a television signal, monthly sales of a corporation, or daily closing prices of a stock market (e.g., the Dow Jones averages). In all these examples, the signals are functions of the independent variable time. This is not always the case, however. When an electrical charge is distributed over a body, for instance, the signal is the charge dens

In [13]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
vector_store = Chroma.from_documents(split_docs, embeddings, persist_directory="./new_test")

In [14]:
# https://github.com/langchain-ai/langchain/issues/5462
def ask_with_memory(vector_store, question, chat_history=[]):
    from langchain.chains import ConversationalRetrievalChain
    from langchain_openai import ChatOpenAI
    from langchain_core.prompts import ChatPromptTemplate
    from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate
    from langchain_core.messages import SystemMessage
    
    llm = ChatOpenAI(openai_api_key=os.environ.get('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0)
    # Define the system message template
    system_template = """
            You are a Artificial Intelligence Tutor of Signal ans System subject.
            You task is to help student to understand Signal ans System subject concept 
            and help them to solve problem step by step. You should not solve the problem directly,
            rather you help them to learn and solve the problem gradually and also make sure that 
            your response are in latex. If you cannot find the answer from the pieces of context, 
            just say that you don't know, don't try to make up an answer.
            -------------------------------------------
            {context}
        """

    # Create the chat prompt templates
    messages = [
        SystemMessagePromptTemplate.from_template(system_template),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
    qa_prompt = ChatPromptTemplate.from_messages(messages)
    
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})
    crc = ConversationalRetrievalChain.from_llm(
        llm=llm, 
        retriever=retriever, 
        # verbose=True,
        return_source_documents=True, 
        combine_docs_chain_kwargs={"prompt": qa_prompt}
    )
    
    result = crc({'question': question, 'chat_history': chat_history})
    chat_history.append((question, result['answer']))
    
    return result, chat_history

In [15]:
## asking with memory
chat_history = []
question = '$y(t) = x(2t)$. Is it a linear system?'
# question = 'what is system and signal'
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print(result['answer'])
# print(chat_history)

c:\Users\shayn\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


To determine if the system described by $y(t) = x(2t)$ is linear, we need to check if it satisfies the properties of linearity.

A system is linear if it satisfies two properties: homogeneity and additivity.

1. **Homogeneity**: For a system to be linear, it must satisfy the property of homogeneity. This property states that if the input is scaled by a constant, then the output is also scaled by the same constant.

Let's check if the system $y(t) = x(2t)$ satisfies homogeneity:

Consider an input signal $x_1(t)$ and its corresponding output $y_1(t)$:
\[ y_1(t) = x_1(2t) \]

Now, let's scale the input signal by a constant $a$ to get a new input signal $x_2(t) = ax_1(t)$. The corresponding output $y_2(t)$ for this new input is:
\[ y_2(t) = x_2(2t) = a x_1(2t) = a y_1(t) \]

Since the output is scaled by the same constant $a$ as the input, the system satisfies homogeneity.

2. **Additivity**: The system should also satisfy the property of additivity. This property states that the response

In [16]:
from IPython.display import display, Latex, Markdown
display(Latex(result['answer']))

<IPython.core.display.Latex object>

In [17]:
from IPython.display import display, Markdown
display(Markdown(result['answer']))

To determine if the system described by $y(t) = x(2t)$ is linear, we need to check if it satisfies the properties of linearity.

A system is linear if it satisfies two properties: homogeneity and additivity.

1. **Homogeneity**: For a system to be linear, it must satisfy the property of homogeneity. This property states that if the input is scaled by a constant, then the output is also scaled by the same constant.

Let's check if the system $y(t) = x(2t)$ satisfies homogeneity:

Consider an input signal $x_1(t)$ and its corresponding output $y_1(t)$:
\[ y_1(t) = x_1(2t) \]

Now, let's scale the input signal by a constant $a$ to get a new input signal $x_2(t) = ax_1(t)$. The corresponding output $y_2(t)$ for this new input is:
\[ y_2(t) = x_2(2t) = a x_1(2t) = a y_1(t) \]

Since the output is scaled by the same constant $a$ as the input, the system satisfies homogeneity.

2. **Additivity**: The system should also satisfy the property of additivity. This property states that the response to the sum of two inputs should be the sum of the responses to each input separately.

Let's check if the system $y(t) = x(2t)$ satisfies additivity:

Consider two input signals $x_1(t)$ and $x_2(t)$ with corresponding outputs $y_1(t)$ and $y_2(t)$:
\[ y_1(t) = x_1(2t) \]
\[ y_2(t) = x_2(2t) \]

Now, consider the input signal $x(t) = x_1(t) + x_2(t)$. The corresponding output $y(t)$ for this new input is:
\[ y(t) = x(2t) = (x_1(t) + x_2(t))(2t) = x_1(2t) + x_2(2t) = y_1(t) + y_2(t) \]

Since the response to the sum of two inputs is the sum of the responses to each input separately, the system also satisfies additivity.

Therefore, the system described by $y(t) = x(2t)$ is indeed a linear system as it satisfies both homogeneity and additivity properties.